![Dataging](https://raw.githubusercontent.com/dataging/public-resources/61263724aea5476ba5ebf38478beada519091957/logodataging.png)
# Pandas y Bases de Datos MySQL
En esta demostración veremos como conectar a una base de datos MySQL, leer datos de tablas, y ejecutar consultas utilizando el conector de MySQL

In [10]:
import os
from dotenv import load_dotenv

load_dotenv() #lee las variables de entorno del archivo .env

servidor = os.getenv("SERVIDOR_MYSQL")
usuario = os.getenv("USUARIO_MYSQL")
password = os.getenv("PASSWORD_MYSQL")

import mysql.connector
try:
    cnx = mysql.connector.connect(user=usuario, password=password,
                              host=servidor, database='sakila') # Definimos la cadena de conexión de la base de datos e intentamos conectar
    cursor=cnx.cursor() #Creamos un cursor para poder ejecutar consultas
    cursor.execute("SELECT * FROM actor") #Ejecutamos la consulta SQL
    resultado = cursor.fetchall() # Obtenemos todas las filas de la consulta
    cursor.close() 
    cnx.close()
except mysql.connector.Error as err:
    print("Error conectando a la base de datos " + err)
    


In [11]:
import pandas as pd
#Convertimos el resultado en un DataFrame de Pandas seleccionando solo cuatro columnas
df = pd.DataFrame(resultado, columns=['actor_id','first_name','last_name','timestamp']) 
df.head() # Mostramos las cinco primeras filas del DataFrame

,actor_id,first_name,last_name,timestamp
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [6]:
#Convertimos el resultado en un DataFrame de Pandas seleccionando todas las columnas
df_actor = pd.DataFrame(resultado, columns=cursor.column_names) 
df_actor.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [9]:
# Como alternativa podemos utilizar el método read_sql de Pandas para ejecutar la consulta y obtener el resultado en un DataFrame
#Obviamente la conexión debe de estar abierta. Debemos de acordarnos de cerrarla cuando finalicemos
try:
    cnx = mysql.connector.connect(user=usuario, password=password,
                              host=servidor, database='sakila')
except mysql.connector.Error as err:
    print("Error conectando a la base de datos " + err)

df = pd.read_sql("SELECT * FROM actor", cnx)
df.head()

C:\Users\anton\AppData\Local\Temp\ipykernel_20668\3482251282.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM actor", cnx)


,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [6]:
def leer_tabla(tabla):
    """ Función para leer una tabla de la base de datos sakila
    y devolver un Dataframe
    
    Parámetros:
    tabla -- Nombre de la tabla a leer
    
    Ejemplo:
        leer_tabla('actor')

    """
    try:
        cnx = mysql.connector.connect(user=usuario, password=password,
                                  host=servidor, database='sakila') # Definimos la cadena de conexión de la base de datos e intentamos conectar
        cursor=cnx.cursor() #Creamos una cursos para poder ejecutar consultas
        cursor.execute(f"SELECT * FROM {tabla}") #Ejecutamos la consulta SQL
        resultado = cursor.fetchall() # Obtenemos todas las filas de la consulta
        cursor.close() 
        cnx.close()
    except mysql.connector.Error as err:
        print("Error conectando a la base de datos " + err)

    
    return pd.DataFrame(resultado, columns=cursor.column_names) #Convertimos el resultado en un DataFrame de Pandas seleccionando todas las columnas y lo devolvemos

In [7]:
df_films = leer_tabla('film')
df_films.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,None,3,4.99,48,12.99,G,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,None,7,2.99,50,18.99,NC-17,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,None,5,2.99,117,26.99,G,"{Behind the Scenes, Commentaries}",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,None,6,2.99,130,22.99,G,{Deleted Scenes},2006-02-15 05:03:42


In [8]:
df_film_actor = leer_tabla('film_actor')
df_film_actor.head()

,actor_id,film_id,last_update
0,1,1,2006-02-15 05:05:03
1,1,23,2006-02-15 05:05:03
2,1,25,2006-02-15 05:05:03
3,1,106,2006-02-15 05:05:03
4,1,140,2006-02-15 05:05:03


In [9]:
df = df_film_actor.merge(df_actor, on='actor_id' ).merge(df_films, on='film_id')
df.head()

,actor_id,film_id,last_update_x,first_name,last_name,last_update_y,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,1,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42
1,1,23,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ANACONDA CONFESSIONS,A Lacklusture Display of a Dentist And a Denti...,2006,1,None,3,0.99,92,9.99,R,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42
2,1,25,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ANGELS LIFE,A Thoughtful Display of a Woman And a Astronau...,2006,1,None,3,2.99,74,15.99,G,{Trailers},2006-02-15 05:03:42
3,1,106,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,BULWORTH COMMANDMENTS,A Amazing Display of a Mad Cow And a Pioneer w...,2006,1,None,4,2.99,61,14.99,G,{Trailers},2006-02-15 05:03:42
4,1,140,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,CHEAPER CLYDE,A Emotional Character Study of a Pioneer And a...,2006,1,None,6,0.99,87,23.99,G,"{Behind the Scenes, Commentaries, Trailers}",2006-02-15 05:03:42


In [12]:
# Hacemos join de los tres dataframes que hemos cargado

df = pd.merge(pd.merge(df_film_actor, df_actor, on='actor_id'), df_films, on='film_id')
# df = pd.merge(pd.merge(df_film_actor, df_actor, how='inner',left_on='actor_id', right_on='actor_id'), df_films, on='film_id')
df.head()

,actor_id,film_id,last_update_x,first_name,last_name,last_update_y,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,1,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42
1,1,23,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ANACONDA CONFESSIONS,A Lacklusture Display of a Dentist And a Denti...,2006,1,None,3,0.99,92,9.99,R,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42
2,1,25,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ANGELS LIFE,A Thoughtful Display of a Woman And a Astronau...,2006,1,None,3,2.99,74,15.99,G,{Trailers},2006-02-15 05:03:42
3,1,106,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,BULWORTH COMMANDMENTS,A Amazing Display of a Mad Cow And a Pioneer w...,2006,1,None,4,2.99,61,14.99,G,{Trailers},2006-02-15 05:03:42
4,1,140,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,CHEAPER CLYDE,A Emotional Character Study of a Pioneer And a...,2006,1,None,6,0.99,87,23.99,G,"{Behind the Scenes, Commentaries, Trailers}",2006-02-15 05:03:42


In [13]:
# Renombramos las columnas que tenían el mismo nombre y les ha puesto un sufijo
df.rename(columns={'last_update_x':'last_update_actor', 'last_update_y':'last_update_film' , 'last_update':'last_update_film_actor'}, inplace=True)
df.head()

,actor_id,film_id,last_update_actor,first_name,last_name,last_update_film,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update_film_actor
0,1,1,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,None,6,0.99,86,20.99,PG,"{Deleted Scenes, Behind the Scenes}",2006-02-15 05:03:42
1,1,23,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ANACONDA CONFESSIONS,A Lacklusture Display of a Dentist And a Denti...,2006,1,None,3,0.99,92,9.99,R,"{Deleted Scenes, Trailers}",2006-02-15 05:03:42
2,1,25,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,ANGELS LIFE,A Thoughtful Display of a Woman And a Astronau...,2006,1,None,3,2.99,74,15.99,G,{Trailers},2006-02-15 05:03:42
3,1,106,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,BULWORTH COMMANDMENTS,A Amazing Display of a Mad Cow And a Pioneer w...,2006,1,None,4,2.99,61,14.99,G,{Trailers},2006-02-15 05:03:42
4,1,140,2006-02-15 05:05:03,PENELOPE,GUINESS,2006-02-15 04:34:33,CHEAPER CLYDE,A Emotional Character Study of a Pioneer And a...,2006,1,None,6,0.99,87,23.99,G,"{Behind the Scenes, Commentaries, Trailers}",2006-02-15 05:03:42


In [17]:
def ejecutar_consulta(consulta):
    """ Función para ejecutar una consulta en la base de datos sakila
    y devolver un Dataframe
    
    Parámetros:
    consulta -- Consulta SQL a ejecutar
    
    Ejemplo:
        ejecutar_consulta('SELECT * FROM actor')

    """
    try:
        cnx = mysql.connector.connect(user=usuario, password=password,
                                  host=servidor, database='sakila') # Definimos la cadena de conexión de la base de datos e intentamos conectar
        cursor=cnx.cursor() #Creamos una cursos para poder ejecutar consultas
        cursor.execute(consulta) #Ejecutamos la consulta SQL
        resultado = cursor.fetchall() # Obtenemos todas las filas de la consulta
        cursor.close() 
        cnx.close()
    except mysql.connector.Error as err:
        print("Error conectando a la base de datos " + str(err))

    
    return pd.DataFrame(resultado, columns=cursor.column_names) #Convertimos el resultado en un DataFrame de Pandas seleccionando todas las columnas y lo devolvemos

In [20]:
consulta="""select first_name, last_name, title, description, release_year, length, rating, d.name as idiom 
    from actor a inner join film_actor b 
        on a.actor_id = b.actor_id 
    inner join film c 
        on b.film_id = c.film_id 
    inner join language d 
        on c.language_id = d.language_id"""

df_consulta = ejecutar_consulta(consulta)
df_consulta.head()

,first_name,last_name,title,description,release_year,length,rating,idiom
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,English
1,CHRISTIAN,GABLE,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,English
2,LUCILLE,TRACY,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,English
3,SANDRA,PECK,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,English
4,JOHNNY,CAGE,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,86,PG,English
